In this notebook we will load and use a masked language model. This kind of a model predicts which words would replace masked words in a sentence. Learn more about the Fill-Mask task <a href="https://huggingface.co/tasks/fill-mask" target="_blank" rel="noopener">here</a>.

We will be running SQL queries using <a href="https://jupysql.ploomber.io/en/latest/quick-start.html" target="_blank" rel="noopener"> JupySQL</a> SQL Magic.

Prior to using this notebook one needs to complete the follow steps:
1. [Create the database schema](../setup_db.ipynb).
2. [Initialize the Transformer Extension](te_init.ipynb).

In [1]:
# TODO: Move this to a separate configuration notebook. Here we just need to load this configuration from a store.
from dataclasses import dataclass

@dataclass
class SandboxConfig:
    EXTERNAL_HOST_NAME = "192.168.124.93"
    HOST_PORT = "8888"

    @property
    def EXTERNAL_HOST(self):
        return f"""{self.EXTERNAL_HOST_NAME}:{self.HOST_PORT}"""

    USER = "sys"
    PASSWORD = "exasol"
    BUCKETFS_PORT = "6666"
    BUCKETFS_USER = "w"
    BUCKETFS_PASSWORD = "write"
    BUCKETFS_USE_HTTPS = False
    BUCKETFS_SERVICE = "bfsdefault"
    BUCKETFS_BUCKET = "default"

    @property
    def EXTERNAL_BUCKETFS_HOST(self):
        return f"""{self.EXTERNAL_HOST_NAME}:{self.BUCKETFS_PORT}"""

    @property
    def BUCKETFS_URL_PREFIX(self):
        return "https://" if self.BUCKETFS_USE_HTTPS else "http://"

    @property
    def BUCKETFS_PATH(self):
        # Filesystem-Path to the read-only mounted BucketFS inside the running UDF Container
        return f"/buckets/{self.BUCKETFS_SERVICE}/{self.BUCKETFS_BUCKET}"

    SCRIPT_LANGUAGE_NAME = "PYTHON3_60"
    UDF_FLAVOR = "python3-ds-EXASOL-6.0.0"
    UDF_RELEASE= "20190116"
    UDF_CLIENT = "exaudfclient" # or for newer versions of the flavor exaudfclient_py3
    SCHEMA = "IDA"

    @property
    def SCRIPT_LANGUAGES(self):
        return f"""{self.SCRIPT_LANGUAGE_NAME}=localzmq+protobuf:///{self.BUCKETFS_SERVICE}/
            {self.BUCKETFS_BUCKET}/{self.UDF_FLAVOR}?lang=python#buckets/{self.BUCKETFS_SERVICE}/
            {self.BUCKETFS_BUCKET}/{self.UDF_FLAVOR}/exaudf/{self.UDF_CLIENT}""";

    @property
    def connection_params(self):
        return {"dns": self.EXTERNAL_HOST, "user": self.USER, "password": self.PASSWORD, "compression": True}

    @property
    def params(self):
        return {
            "script_languages": self.SCRIPT_LANGUAGES,
            "script_language_name": self.SCRIPT_LANGUAGE_NAME,
            "schema": self.SCHEMA,
            "BUCKETFS_PORT": self.BUCKETFS_PORT,
            "BUCKETFS_USER": self.BUCKETFS_USER,
            "BUCKETFS_PASSWORD": self.BUCKETFS_PASSWORD,
            "BUCKETFS_USE_HTTPS": self.BUCKETFS_USE_HTTPS,
            "BUCKETFS_BUCKET": self.BUCKETFS_BUCKET,
            "BUCKETFS_PATH": self.BUCKETFS_PATH
        }

    # Name of the BucketFS connection
    BFS_CONN = 'MyBFSConn'

    # Name of a sub-directory of the bucket root
    BFS_DIR = 'my_storage'

    # We will store all models in this sub-directory at BucketFS
    TE_MODELS_DIR = 'models'
    
    # We will save cached model in this sub-directory relative to the current directory on the local machine.
    TE_MODELS_CACHE_DIR = 'models_cache'

    @property
    def WEBSOCKET_URL(self):
        return f"exa+websocket://{self.USER}:{self.PASSWORD}@{self.EXTERNAL_HOST}/{self.SCHEMA}?SSLCertificate=SSL_VERIFY_NONE"

conf = SandboxConfig()

First let's bring up the JupySQL and connect to the database via the SQLAlchemy. Please refer to the documentation in the <a href="https://github.com/exasol/sqlalchemy-exasol" target="_blank" rel="noopener">sqlalchemy-exasol</a> for details on how to connect to the database using Exasol SQLAlchemy driver.

In [2]:
from sqlalchemy import create_engine

engine = create_engine(conf.WEBSOCKET_URL)

%load_ext sql
%sql engine

Now we will download a model from the Huggingface Hub and put into the BucketFS.

There are two ways of doing this.
1. Using the `TE_MODEL_DOWNLOADER_UDF` UDF.
2. Downloading a model to a local drive and subsequently uploading in into the BucketFS using a CLI.

In this notebook we will use the second method.

To demonstrate finding of a masked word task we will use a [RadBERT model](https://huggingface.co/StanfordAIMI/RadBERT) which was pre-trained on radiology reports. This is a public model, therefore the last parameter - the name of the Huggingface token connection - can be an empty string.

In [11]:
# This is the name of the model at the Huggingface Hub
MODEL_NAME = 'StanfordAIMI/RadBERT'

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=conf.TE_MODELS_CACHE_DIR)
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME, cache_dir=conf.TE_MODELS_CACHE_DIR)

Now we can upload the model into the BucketFS using a command line. Unfortunately we cannot tell exactly when this process has finished. Notebook's hourglass may not be a reliable indicator. BucketFS will still be doing some work when the call issued by the notebook returns. Please wait for few moments after that, before querying the model.

In [13]:
upload_command = f"""python -m exasol_transformers_extension.upload_model \
    --bucketfs-name {conf.BUCKETFS_SERVICE} \
    --bucketfs-host {conf.EXTERNAL_HOST_NAME} \
    --bucketfs-port {conf.BUCKETFS_PORT} \
    --bucketfs-user {conf.BUCKETFS_USER} \
    --bucketfs-password {conf.BUCKETFS_PASSWORD} \
    --bucket {conf.BUCKETFS_BUCKET} \
    --path-in-bucket {conf.BFS_DIR} \
    --model-name {MODEL_NAME}  \
    --sub-dir {conf.TE_MODELS_DIR} \
    --local-model-path {conf.TE_MODELS_CACHE_DIR}
    """
!{upload_command}

Let's see if the model can find a masked word in an instruction usually given to a patient when radiographer is doing her chest X-ray.

In [3]:
# This is a sentence with a masked word that will be given to the model.
MY_TEXT = 'Take a deep [MASK] and hold it'

# Make sure our text can be used in an SQL statement.
MY_TEXT = MY_TEXT.replace("'", "''")

# We will collect 5 best answers.

In [ ]:
%%sql
WITH MODEL_OUTPUT AS
(
    SELECT TE_FILLING_MASK_UDF(
        NULL,
        '{{conf.BFS_CONN}}',
        NULL,
        '{{conf.TE_MODELS_DIR}}',
        '{{MODEL_NAME}}',
        '{{MY_TEXT}}',
        5
    )
)
SELECT filled_text, score, error_message FROM MODEL_OUTPUT ORDER BY SCORE DESC